In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn

from training import train_model
from data import get_dataloaders
from models import *

from transnet import Transformer

import os

device = torch.device("cuda")


torch.use_deterministic_algorithms(True)

# Variable must be set to allow deterministic
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

SEED = 777

torch_rng = torch.manual_seed(SEED)
np_rng = np.random.default_rng(SEED)

np.random.seed(SEED)

import sys

EPOCHS = 100
BATCH_SIZE = 200 #
PREDICTION_TARGET = "sbox"
TARGET_BYTE_IDX = 1
LEARNING_RATE = 0.00001

TRACE_INTERVAL_START = 900
TRACE_INTERVAL_END = 1200

INPUT_LENGTH = TRACE_INTERVAL_END - TRACE_INTERVAL_START



train_loader, val_loader, _ = get_dataloaders(
    BATCH_SIZE,
    PREDICTION_TARGET,
    TARGET_BYTE_IDX,
    TRACE_INTERVAL_START,
    TRACE_INTERVAL_END
)

In [8]:
from importlib import reload

import transnet
transnet = reload(transnet)

model = transnet.Transformer(1,8,4,2,16,0.,0.,256,11,2,
                    clamp_len=300,
                    untie_r=True,
                    smooth_pos_emb=False,
                    untie_pos_emb=True
)

model_name = f"fixslice-{PREDICTION_TARGET}-byte{TARGET_BYTE_IDX}-zhang-{TRACE_INTERVAL_START}_{TRACE_INTERVAL_END}"

model = model.to(device)



# Zhang 2019
optimizer = optim.Adam(model.parameters(), lr=0.00001)
loss_function = nn.CrossEntropyLoss()

try:
    train_model(
        model,
        optimizer,
        loss_function,
        train_loader,
        val_loader,
        model_name,
        ([],[]),
        EPOCHS,
        PREDICTION_TARGET,
    )

except KeyboardInterrupt:
    print("Cancelled")

training:   6%|▌         | 148/2500 [00:05<01:19, 29.53batch/s]

Cancelled
